## Collect data

In [ ]:
!kaggle competitions download -c airbus-ship-detection

In [ ]:
!mv *.zip *.csv data/

In [ ]:
!unzip data/train_v2.zip -d data/train
!unzip data/test_v2.zip -d data/test
!unzip data/train_ship_segmentations_v2.csv.zip

In [ ]:
!rm data/*.zip
!ls data/

## Process data

In [ ]:
import pandas as pd
from pathlib import Path
from fastai.vision import *
%matplotlib inline

# Data locations
path = Path("data/")

In [ ]:
df = pd.read_csv(path/"sample_submission_v2.csv")
df.head()

In [ ]:
!chmod a+r data/train_ship_segmentations_v2.csv
!chmod u+w data/train_ship_segmentations_v2.csv

In [ ]:
rle_masks = pd.read_csv(path/"train_ship_segmentations_v2.csv")
rle_masks.head()

In [ ]:
rle_masks = rle_masks.fillna(0)
rle_masks.to_csv(path/"train_ship_segmentations_v2_filtered.csv")

In [ ]:
rle_masks.head()

In [ ]:
imsize = (768, 768)
open_mask_rle(rle_masks["EncodedPixels"][2], shape=imsize).shape

In [ ]:
open_mask_rle(rle_masks["EncodedPixels"][2], shape=imsize)

In [ ]:
def open_mask_rle_custom(mask_rle, shape):
    import pdb
    pdb.set_trace()
    return open_mask_rle(mask_rle, shape)

In [ ]:
# tfms = get_transforms(flip_vert=True)
# use label from df, + setattr mask opener to open mask rle
# possibly needs train data to be opened with df as well
test = SegmentationItemList.from_csv(path, folder="test", csv_name="sample_submission_v2.csv")
train = SegmentationItemList.from_csv(path, folder="train", csv_name="train_ship_segmentations_v2_filtered.csv")
setattr(train, "open", partial(open_mask_rle, shape=(768, 768)))

In [ ]:
type(train)

In [ ]:
train = (train
    .split_by_rand_pct(0.3)
#     .setattr(open, open_mask_rle)
    .label_from_df(classes=["no ship", "ship"])
#     .mask_opener(open_mask_rle)
    .add_test(test)
    .transform(get_transforms(flip_vert=True), tfm_y=True, size=128)
    .databunch(path=".", bs=64)
    .normalize(imagenet_stats)
        )